In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image_dataset_from_directory
import time
import pathlib

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers


def make_generator_model():
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=(100,)),
        layers.Reshape((1, 1, 100)),

        layers.Conv2DTranspose(2048, (4, 4), strides=(1, 1), padding='valid', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(1024, (4, 4), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

def make_discriminator_model():
    model = tf.keras.Sequential([
        layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same', input_shape=(128, 128, 3), use_bias=False),
        layers.LeakyReLU(0.2),

        layers.Conv2D(256, (4, 4), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2D(512, (4, 4), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2D(1024, (4, 4), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2D(2048, (4, 4), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model


# Create the models
generator = make_generator_model()
discriminator = make_discriminator_model()


In [ ]:
generator.summary()

In [ ]:
discriminator.summary()

In [ ]:
# Define the loss and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5, beta_2=0.999)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5, beta_2=0.999)

In [ ]:
# Load and prepare the dataset
BATCH_SIZE = 32
EPOCHS = 500
noise_dim = 100
num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, noise_dim])

# Directory containing your data
data_dir = '/content/drive/MyDrive/ML_Pro1/training/Loc'

def load_and_preprocess_data():
    # Assuming your directory structure is like: training/center/*.png
    data_dir = pathlib.Path("/content/drive/MyDrive/ML_Pro1/training/Loc")
    # Count the total images
    image_count = len(list(data_dir.glob('*.png')))
    print(f"Total images: {image_count}")

    # Create a dataset from the file paths
    list_ds = tf.data.Dataset.list_files(str(data_dir/'*'))

    # Define a function to load and preprocess each image
    def parse_image(file_path):
        # Load the raw data from the file as a string
        img = tf.io.read_file(file_path)
        # Convert the compressed string to a 3D uint8 tensor
        img = tf.image.decode_png(img, channels=3)
        # Resize the image to the desired size
        img = tf.image.resize(img, [128, 128])
        # Normalize the image data to [0, 1] range
        img = (img - 127.5) / 127.5
        return img

    # Use the `map` method to apply this transformation
    images_ds = list_ds.map(parse_image)

    # Apply the batching
    images_ds = images_ds.batch(BATCH_SIZE)

    return images_ds

In [ ]:
# Perform a forward pass with the generator and discriminator to initialize variables
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)
decision = discriminator(generated_image)
gen_loss_epoch = []
disc_loss_epoch = []
# Training loop
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        gen_loss_list = []
        disc_loss_list = []

        for image_batch in dataset:
            t_gen_loss, t_disc_loss = train_step(image_batch)
            gen_loss_list.append(t_gen_loss)
            disc_loss_list.append(t_disc_loss)

        # Saving (checkpoint) the model every 50 epochs
        if (epoch + 1) % 50 == 0:
            generator.save(f'LocGenerator_epoch_{epoch+1}.h5')  # Save the generator model

        # Save the losses for this epoch
        gen_loss_epoch.append(np.mean(gen_loss_list))
        disc_loss_epoch.append(np.mean(disc_loss_list))

        print(f'Time for epoch {epoch + 1} is {time.time()-start} sec')

    # Save the losses after training
    np.save('generator_losses_Loc.npy', gen_loss_epoch)
    np.save('discriminator_losses_Loc.npy', disc_loss_epoch)


In [ ]:
# Load the dataset
train_dataset = load_and_preprocess_data()

Total images: 1620


In [ ]:
train_dataset

<_BatchDataset element_spec=TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None)>

In [ ]:
# Train the model
train(train_dataset, EPOCHS)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load the losses
generator_losses = np.load('generator_losses_Loc.npy')
discriminator_losses = np.load('discriminator_losses_Loc.npy')

# Generate x values (epochs)
epochs = np.arange(1, len(generator_losses) + 1)

# Plot the losses
plt.figure(figsize=(10, 5))
plt.plot(epochs, generator_losses, label='Generator Loss', color='blue')
plt.plot(epochs, discriminator_losses, label='Discriminator Loss', color='red')
plt.title('Generator and Discriminator Losses for Loc')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('LossGraph_Loc.png')
plt.show()

In [ ]:


#automated image generation
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Define the range of epochs
start_epoch = 50
end_epoch = 501

# Number of images to generate per epoch
num_images_per_epoch = 100

# Loop over the desired range of epochs
for epoch in range(start_epoch,end_epoch, 50):
    # Construct the model file name based on the current epoch
    model_file = f'/content/drive/MyDrive/ML_Pro1/Loc_Models/LocGenerator_epoch_{epoch}.h5'

    # Load the Generator Model
    generator = load_model(model_file)  # Replace with your actual path

    # Create a directory to save the images for each epoch
    epoch_dir = f'/content/drive/MyDrive/ML_Pro1/Loc_data/Loc_{epoch}Epoch_Images'
    os.makedirs(epoch_dir, exist_ok=True)

    # Inner loop for generating images in each epoch
    for i in range(num_images_per_epoch):
        # Generate a random input
        random_input = np.random.randn(1, 100)  # Assuming 100 is the size of your input vector

        # Generate an image
        generated_image = generator.predict(random_input)

        # Define the file path to save the image
        file_path = os.path.join(epoch_dir, f'image_{i}.png')

        # Save the generated image to the specified file path
        tf.keras.preprocessing.image.save_img(file_path, generated_image[0, :, :, :])

    print(f'Generated {num_images_per_epoch} images for epoch {epoch} using {model_file} in {epoch_dir}')

#FID and KID

In [ ]:
import torch
from torchvision.models import inception_v3
from torchvision import transforms
from scipy.linalg import sqrtm
import numpy as np
from torch.nn.functional import adaptive_avg_pool2d
from sklearn.metrics import pairwise_kernels
from torchvision import transforms
from PIL import Image
import glob
import os
import pandas as pd
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn.functional as F

def calculate_frechet_distance(mu1, sigma1, mu2, sigma2):
    # Ensure covariance matrices are 2D arrays
    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    # Calculate FID using the formula
    fid = np.sum((mu1 - mu2)**2) + np.trace(sigma1 + sigma2 - 2 * sqrtm(sigma1 @ sigma2))
    return fid.real

def calculate_kernel_distance(features_real, features_generated, kernel='linear'):
    # Reshape features to 2D arrays
    features_real = np.reshape(features_real, (len(features_real), -1))
    features_generated = np.reshape(features_generated, (len(features_generated), -1))

    # Calculate KID using the specified kernel
    if kernel == 'linear':
        return np.mean(pairwise_kernels(features_real, features_generated, metric='linear'))
    elif kernel == 'rbf':
        return np.mean(pairwise_kernels(features_real, features_generated, metric='rbf'))
    else:
        raise ValueError("Unsupported kernel. Supported kernels are 'linear' and 'rbf'.")

def calculate_activation_statistics(features):
    # Calculate mean and covariance of the features
    mu = np.mean(features, axis=0)
    sigma = np.cov(features, rowvar=False)
    return mu, sigma

def preprocess_image(img_path):
    # Open the image using PIL
    img = Image.open(img_path)

    # Convert to RGB if the image has an alpha channel
    if img.mode == 'RGBA':
        img = img.convert('RGB')

    # Preprocess image to match Inception-v3 requirements
    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = transform(img)
    img = img.unsqueeze(0)  # Add batch dimension
    return img

def calculate_fid_kid(real_images, generated_images, device='cuda', batch_size=50):
    # Load Inception-v3 model
    inception_model = inception_v3(pretrained=True, transform_input=False).to(device)
    inception_model.eval()

    # Calculate Inception-v3 features for real images
    real_features = []
    with torch.no_grad():
        for i in range(0, len(real_images), batch_size):
            batch = real_images[i:i+batch_size].to(device)
            features = inception_model(batch)[0].squeeze().cpu().numpy()
            real_features.append(features)
    real_features = np.concatenate(real_features, axis=0)

    # Calculate Inception-v3 features for generated images
    generated_features = []
    with torch.no_grad():
        for i in range(0, len(generated_images), batch_size):
            batch = generated_images[i:i+batch_size].to(device)
            features = inception_model(batch)[0].squeeze().cpu().numpy()
            generated_features.append(features)
    generated_features = np.concatenate(generated_features, axis=0)

    # Calculate FID and KID scores
    mu_real, sigma_real = calculate_activation_statistics(real_features)
    mu_generated, sigma_generated = calculate_activation_statistics(generated_features)

    fid = calculate_frechet_distance(mu_real, sigma_real, mu_generated, sigma_generated)
    kid = calculate_kernel_distance(real_features, generated_features)

    return fid, kid

def load_and_preprocess_images_from_folder(folder):
    # Load and preprocess only .png files from the specified folder
    file_paths = glob.glob(os.path.join(folder, '*.png'))

    images = []
    for img_path in file_paths:
        try:
            img = preprocess_image(img_path)
            images.append(img)
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

    if not images:
        raise ValueError("No valid images found in the folder.")

    return torch.cat(images, dim=0)  # Use torch.cat instead of torch.stack


def calculate_fid_kid_for_epochs(real_folder, generated_base_folder, epochs, excel_path='FID_KID_scores_Loc.csv'):
    # Check if the Excel file already exists
    if os.path.exists(excel_path):
        # If it exists, read the existing data
        df = pd.read_csv(excel_path)
    else:
        # If it doesn't exist, create an empty DataFrame
        df = pd.DataFrame(columns=['Epoch', 'FID', 'KID'])

    # Iterate over each epoch
    for epoch in epochs:
        epoch_folder_name = f'Loc_{epoch}Epoch_Images'
        generated_images_folder = os.path.join(generated_base_folder, epoch_folder_name)

        # Load real images
        real_images = load_and_preprocess_images_from_folder(real_folder)

        # Load generated images for the current epoch
        try:
            generated_images = load_and_preprocess_images_from_folder(generated_images_folder)
        except ValueError as e:
            print(f"Skipping Epoch {epoch}: {e}")
            continue

        # Calculate FID and KID for the current epoch
        fid, kid = calculate_fid_kid(real_images, generated_images)

        # Append the results to the DataFrame
        new_row = {'Epoch': epoch, 'FID': fid, 'KID': kid}
        df = df.append(new_row, ignore_index=True)

        print(f"Epoch {epoch} - FID: {fid}, KID: {kid}")

    # Save the DataFrame to the CSV file
    df.to_csv(excel_path, index=False)

# Specify the paths for real images and generated images base folder
real_folder = '/content/drive/MyDrive/ML_Pro1/testing/Loc/Loc'
generated_base_folder = '/content/drive/MyDrive/ML_Pro1/Loc_data'

# Specify the range of epochs to calculate
epochs_to_calculate = range(50, 501, 50)  # Adjust as needed

calculate_fid_kid_for_epochs(real_folder, generated_base_folder, epochs_to_calculate)




# PSNR

In [ ]:
import torch
import os
from PIL import Image
import pandas as pd
import numpy as np
import csv

def calculate_psnr(img1, img2, max_value=255):
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    return 20 * torch.log10(max_value / torch.sqrt(mse))


df_psnr = pd.DataFrame(columns=['Epoch', 'Average PSNR'])

avg_psnr = []
epochs = []

def psnr_fn(epoch_number):
  # Define the paths to the generated and real image folders
  fake_image_folder = "/content/drive/MyDrive/ML_Pro1/Loc_data/Loc_"+str(epoch_number)+"Epoch_Images"
  real_image_folder = '/content/drive/MyDrive/ML_Pro1/testing/Loc/Loc'

  psnr_values = []



  fake_image_files = [f for f in os.listdir(fake_image_folder) if f.endswith('.png')]
  real_image_files = [f for f in os.listdir(real_image_folder) if f.endswith('.png')]

  for fake_image_file in fake_image_files:
      for real_image_file in real_image_files:
          fake_image_path = os.path.join(fake_image_folder, fake_image_file)
          real_image_path = os.path.join(real_image_folder, real_image_file)

          # Load images using PIL and convert to PyTorch tensors
          fake_image = torch.tensor(np.array(Image.open(fake_image_path))).float() / 255.0
          real_image = torch.tensor(np.array(Image.open(real_image_path))).float() / 255.0

          # Resize images if needed
          fake_image = torch.nn.functional.interpolate(fake_image.unsqueeze(0), size=(128, 128)).squeeze()
          real_image = torch.nn.functional.interpolate(real_image.unsqueeze(0), size=(128, 128)).squeeze()

          # Calculate PSNR for each color channel
          psnr_channel = []
          for channel in range(3):  # Assuming RGB images
              psnr_value = calculate_psnr(fake_image[channel], real_image[channel], max_value=1.0)
              psnr_channel.append(psnr_value.item())

          # Take the average PSNR across color channels
          average_psnr_channel = sum(psnr_channel) / len(psnr_channel)
          psnr_values.append(average_psnr_channel)



  # Calculate the overall average PSNR
  average_psnr = sum(psnr_values) / len(psnr_values)

  print(f"Avg PSNR at epoch {epoch_number}: {average_psnr:.2f}")


  # Append epochs and avg psnr
  epochs.append(epoch_number)
  avg_psnr.append(average_psnr)


#calling fn over all the epochs
for epoch in [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]:
  psnr_fn(epoch)


# Specify the file name
file_name = '/content/drive/MyDrive/ML_Pro1/output_psnr_Loc.csv'

# Combine the lists into a list of tuples
data = list(zip(epochs, avg_psnr))

# Write the data to a CSV file
with open(file_name, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write the header
    csv_writer.writerow(['Epochs', 'Average PSNR'])

    # Write the data
    csv_writer.writerows(data)

print(f'Data has been saved to {file_name}')

#Inception

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torchvision.models import inception_v3
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image
import os
import numpy as np
import pandas as pd
import re

# Define the Inception model
class InceptionModel(nn.Module):
    def __init__(self):
        super(InceptionModel, self).__init__()
        self.inception_model = inception_v3(pretrained=True, transform_input=False)
        self.inception_model.eval()

    def forward(self, x):
        return self.inception_model(x)[0]

# Define the function to calculate the Inception Score
def calculate_inception_score(images, batch_size=32, resize=True):
    # Set up the Inception model
    inception_model = InceptionModel()

    # Check if there are subdirectories
    subdirectories = [f for f in os.listdir(images) if os.path.isdir(os.path.join(images, f))]

    if subdirectories:
        # Use ImageFolder approach if subdirectories are present
        transform = transforms.Compose([
            transforms.Resize((299, 299)) if resize else transforms.ToTensor(),
            transforms.ToTensor(),
        ])

        dataset = ImageFolder(root=images, transform=transform)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)

        # Compute activations for the generated images
        all_activations = []

        with torch.no_grad():
            for batch in dataloader:
                batch = batch[0]
                activations = inception_model(batch)
                all_activations.append(activations.cpu().numpy())

        all_activations = np.concatenate(all_activations, axis=0)
    else:
        # Direct loading approach if no subdirectories
        # Get the list of image file paths
        image_files = [os.path.join(images, img) for img in os.listdir(images) if img.endswith(('.png', '.jpg', '.jpeg', '.gif'))]

        # Set up data loader for generated images
        transform = transforms.Compose([
            transforms.Resize((299, 299)) if resize else transforms.ToTensor(),
            transforms.ToTensor(),
        ])

        # Load images
        all_images = []
        for img_path in image_files:
            img = Image.open(img_path).convert('RGB')
            img_tensor = transform(img)
            all_images.append(img_tensor)

        if not all_images:
            raise ValueError("No images found in the specified directory.")

        all_activations = torch.stack(all_images)

    # Compute the Inception Score
    scores = []
    split_size = 10
    all_activations_tensor = torch.tensor(all_activations)
    split_activations = torch.split(all_activations_tensor, len(all_activations_tensor) // split_size)

    for activations in split_activations:
        activations = activations.view(-1, activations.shape[-1])
        scores.append(F.softmax(activations, dim=1))

    scores = torch.cat(scores, dim=0)
    kl_divergence = scores * (torch.log(scores) - torch.log(torch.mean(scores, dim=0)))
    final_score = torch.exp(torch.mean(torch.sum(kl_divergence, dim=1)))

    return final_score.item()

# Path to the folder containing generated images for different epochs
generated_images_base_folder = "/content/drive/MyDrive/ML_Pro1/Loc_data"

# CSV file to store Inception Scores
csv_file_path = 'inception_scores_Loc.csv'

# Create an empty DataFrame to store results
df_combined = pd.DataFrame(columns=['Epoch', 'Inception_Score'])

# Iterate over each subfolder corresponding to different epochs
for epoch_folder in sorted(os.listdir(generated_images_base_folder), key=lambda x: int(re.search(r'(\d+)', x).group(0)) if re.search(r'(\d+)', x) else float('inf')):
    epoch_folder_path = os.path.join(generated_images_base_folder, epoch_folder)

    # Check if it is a directory
    if os.path.isdir(epoch_folder_path):
        try:
            # Calculate Inception Score for each epoch
            inception_score = calculate_inception_score(epoch_folder_path)

            # Append the new Inception Score to the DataFrame
            df_combined = pd.concat([df_combined, pd.DataFrame({'Epoch': [epoch_folder], 'Inception_Score': [inception_score]})], ignore_index=True)

        except ValueError as e:
            print(f"Skipping {epoch_folder}: {e}")

# Save to CSV
df_combined.to_csv(csv_file_path, index=False)

print("Inception Scores calculation completed and saved to CSV.")

